In [38]:
from pathlib import Path
import json
import glob
import re
import numpy as np
import pandas as pd


def process_files(globpath):
    '''
    Parameters
    ----------
    globpath : str
        Path with wildcard of json files.

    Returns
    -------
    dictionary of files and paths with parsed names
    '''
    all_files = glob.glob(globpath)
    # print(all_files)
    
    xs = []
    ys = []
    filenames = {}
    
    for file in all_files:
        scores = json.loads(Path(file).read_text()) # load json file as dictionary from path file
        #print(scores)
        #print(file)
        x = re.search("(WP_\d+)", file)[1] # search for the first WP_ and take the first group (WP_\d+)
        y = re.search("_(WP_\d+)", file)[1] # search for the second WP_ and take the first group (WP_\d+) 
        filenames[x + "_" + y]  = file
        
        xs.append(x)
        ys.append(y)
        
    xs = list(set(xs))
    ys = list(set(ys))
    
    xs.sort()
    ys.sort()
    
    return filenames, xs, ys

    
def define_good_tracts(plddt):
    '''
    Parameters
    ----------
    plddt : DataFrame
        dataframe with plddt values
    
    Returns
    -------
    good_tracts : DataFrame
        DataFrame with info about tracts (tracts are consecutive regions with plddt > 70)

    '''
    rolling_score_plddt = plddt[0].rolling(window = 30).mean().to_frame()
    rolling_score_plddt[1] = rolling_score_plddt[0] > 70
    rolling_score_plddt.reset_index(inplace = True)
        
    good_tracts = rolling_score_plddt.loc[rolling_score_plddt[1] == True]        
    good_tracts['shifted_index_f'] = good_tracts['index'].shift(1)
    good_tracts['diff_f']  =  good_tracts['index'] - good_tracts['shifted_index_f']
    good_tracts['spaced_f'] = good_tracts['diff_f'] == 1
    good_tracts['gap_size'] = good_tracts['spaced_f']
    good_tracts['tract_ID'] = good_tracts.loc[good_tracts['gap_size']==False].groupby('gap_size').cumcount() # cumcount() counts the number of consecutive True values 
    good_tracts['tract_ID'] = good_tracts['tract_ID'].fillna(method='ffill') # fill the NaNs with the previous value
    
    return good_tracts
    

def calculate_rectangle_reversed_normalized_mean(pae, start1, end1, start2, end2):
    '''
    Takes two rectangles in pae matrix, normalize score between 0 and 1 (1 being best confidence)
    These rectangles represent the -,- and +,- quadrants, where we expect to see signs of intermolecular PAE scores
    
    Parameters
    ----------
    pae : DataFrame
        pae matrix of scores from 0-30

    start :
        start coordinate of interest of square
        
    end : 
        end coordinate of interest of square 

    Returns
    -------
    mean and std of flattened score matrix in intermolecular region
    '''
    reversed_normalized_pae = (30 - pae)/30
    rec1 = reversed_normalized_pae.iloc[start1:end1, start2:end2]
    rec2 = reversed_normalized_pae.iloc[start2:end2, start1:end1]

    rec1_flat = rec1.to_numpy().flatten()
    rec2_flat = rec2.to_numpy().flatten()
 
    allscores_flat = np.concatenate((np.array(rec1_flat), np.array(rec2_flat)))

    return allscores_flat.mean(), allscores_flat.std()

def scoring(x,y, filenames, df):
    '''
    Parameters
    ----------
    x : str
        processed name of toxin that is a key in dictionary of files (toxin name)

    y : str
        processed name of toxin that is a key in dictionary of files (imm name)
        
    filenames : dict
        dictionary with filenames. x and y are keys to this dict
        
    df : DataFrame
        dataframe with size info

    Returns
    -------
    scoring
    '''
    
    out = []

    # load json, save in df
    scores = json.loads(Path(filenames[x + "_" + y]).read_text())
    plddt = pd.DataFrame(scores['plddt'])
    pae = pd.DataFrame(scores['pae'])
    
    # calculate labels and lengths
    label = x + " and " + y
    print(label)
    length_x = int(df.loc[df['Gene ID'] == x+'.1']['Amino Acid Sequence Length (aa)'].iloc[0])
    length_y = int(df.loc[df['Gene ID'] == y+'.1']['Amino Acid Sequence Length (aa)'].iloc[0])
    length_together = length_x + length_y

    #are they expected?
    
    #digitsX = [int(s) for s in x.split() if s.isdigit()]
    #digitsY = [int(s) for s in y.split() if s.isdigit()]
    #print(digitsX, digitsY)
    #expected = abs(digitsX[0] - digitsY[0]) <= 1
    digitsX = str(1)
    digitsY = str(1)
    expected = None
    print(expected)
    # call function to find good tracts
    good_tracts_x = define_good_tracts(plddt.iloc[range(0, length_x)])
    good_tracts_y = define_good_tracts(plddt.iloc[range(length_x, length_together)])
    
    
    if (not good_tracts_x.empty) and (not good_tracts_y.empty):
    
        # find start coordinates of good tracts                        
        starts_x = good_tracts_x.groupby('tract_ID')['index'].min().to_frame().rename(columns = {'index' : 'start_coord'})
        starts_y = good_tracts_y.groupby('tract_ID')['index'].min().to_frame().rename(columns = {'index' : 'start_coord'})
        
        # calculate end coordinates of good tracts
        starts_x['end_coord'] = starts_x['start_coord'].shift(-1).fillna(good_tracts_x['index'].iloc[-1]).astype(int)
        starts_y['end_coord'] = starts_y['start_coord'].shift(-1).fillna(good_tracts_y['index'].iloc[-1]).astype(int)
        
        for i in starts_x.index.unique():
            
            start_coord_x_tract = int(starts_x.iloc[int(i)]['start_coord']) 
            end_coord_x_tract = int(starts_x.iloc[int(i)]['end_coord'])
            
            for j in starts_y.index.unique():
    
                start_coord_y_tract = int(starts_y.iloc[int(j)]['start_coord'])
                end_coord_y_tract = int(starts_y.iloc[int(j)]['end_coord'])
                
                
                
                intermolecular_score = calculate_rectangle_reversed_normalized_mean(pae, start_coord_x_tract, 
                                                                                     end_coord_x_tract, 
                                                                                     start_coord_y_tract, 
                                                                                     end_coord_y_tract)
                
                out.append([label, 
                            i, 
                            j, 
                            start_coord_x_tract, 
                            end_coord_x_tract, 
                            start_coord_y_tract, 
                            end_coord_y_tract,
                            intermolecular_score[0],
                            intermolecular_score[1],
                            expected,
                            digitsX[0],
                            digitsY[0]])
                
    else:
        out.append([label, 
                    None, 
                    None, 
                    None, 
                    None, 
                    None, 
                    None,
                    None,
                    None,
                    expected,
                    digitsX[0],
                    digitsY[0]])
                
    outframe = pd.DataFrame(out, columns = ['protein_names', 
                                            'tract ID top left', 
                                            'tract ID bottom right', 
                                            'start_coord_top_left_tract', 
                                            'end_coord_top_left_tract', 
                                            'start_coord_bottom_right_tract', 
                                            'end_coord_bottom_right_tract',
                                            'mean_intermolecular_PAE',
                                            'std_dev_intermolecular_PAE',
                                            'expected',
                                            'x_name',
                                            'y_name'])
                                
    return outframe
            
    
########################


In [39]:
# PREPARE DATA

pot_new_pims = pd.read_excel(r"C:\Users\zinge\Documents\BSc\Y3\Research Exercise (71260)\pot_new_pims.xlsx")

folded_pairs = pot_new_pims.loc[pot_new_pims['pass_foldseek'] == 'YES']
folded_pairs_lenghts = folded_pairs.loc[:, ['tox_refseq', 'new_imm_refseq', 'tox_length', 'imm_length']]

# reorder the data to a two-column format: 'Gene ID' and 'Amino Acid Sequence Length (aa)'

folded_pairs_lenghts_formated = pd.DataFrame(columns=['Gene ID', 'Amino Acid Sequence Length (aa)'])
folded_pairs_lenghts_formated['Gene ID'] = pd.concat([folded_pairs_lenghts['tox_refseq'], folded_pairs_lenghts['new_imm_refseq']])
folded_pairs_lenghts_formated['Amino Acid Sequence Length (aa)'] = pd.concat([folded_pairs_lenghts['tox_length'], folded_pairs_lenghts['imm_length']])
folded_pairs_lenghts_formated.reset_index(inplace = True, drop = True)


In [40]:
# Import size data
df = folded_pairs_lenghts_formated

globpath = r"C:\Users\zinge\Documents\BSc\Y3\Research Exercise (71260)\Folded Models\pot_new_pims_json\*rank_1*.json"
filenames, xs, ys = process_files(globpath)
for file in filenames:
    print(file)
final = [] # list of dataframes with scores for each protein pair

for file in filenames:
    x = re.search("(WP_\d+)_", file)[1]
    y = re.search("_(WP_\d+)", file)[1]       
    final.append(scoring(x, y, filenames, df))
        
final_df = pd.concat(final)
final_df.reset_index(drop = True, inplace = True)
final_df.fillna(value = 0, inplace = True)

idx = final_df.groupby("protein_names")['mean_intermolecular_PAE'].idxmax() # idx is the index of the maxima (the best intermolecular PAE score) of each group (protein pair) 

filtered_df = final_df.loc[idx]
filtered_df.sort_values(['x_name', 'expected'], inplace = True)

print(filtered_df)

filtered_df.to_csv("scores_maxima.csv", index = False)

WP_149191159_WP_149191160
WP_185560005_WP_185560006
WP_199404901_WP_199404900
WP_199405059_WP_199405058
WP_205420459_WP_055004244
WP_226888708_WP_226003473
WP_228302989_WP_223575842
WP_228474820_WP_185306028
WP_233679651_WP_061090952
WP_235624429_WP_012296277
WP_243820847_WP_243820848
WP_275719485_WP_275719486
WP_149191159 and WP_149191160
None
WP_185560005 and WP_185560006
None
WP_199404901 and WP_199404900
None


C:\Users\zinge\AppData\Local\Temp\ipykernel_15884\1017998909.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  good_tracts['shifted_index_f'] = good_tracts['index'].shift(1)
C:\Users\zinge\AppData\Local\Temp\ipykernel_15884\1017998909.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  good_tracts['diff_f']  =  good_tracts['index'] - good_tracts['shifted_index_f']
C:\Users\zinge\AppData\Local\Temp\ipykernel_15884\1017998909.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

WP_199405059 and WP_199405058
None
WP_205420459 and WP_055004244
None
WP_226888708 and WP_226003473
None


C:\Users\zinge\AppData\Local\Temp\ipykernel_15884\1017998909.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  good_tracts['shifted_index_f'] = good_tracts['index'].shift(1)
C:\Users\zinge\AppData\Local\Temp\ipykernel_15884\1017998909.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  good_tracts['diff_f']  =  good_tracts['index'] - good_tracts['shifted_index_f']
C:\Users\zinge\AppData\Local\Temp\ipykernel_15884\1017998909.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

WP_228302989 and WP_223575842
None
WP_228474820 and WP_185306028
None
WP_233679651 and WP_061090952
None


C:\Users\zinge\AppData\Local\Temp\ipykernel_15884\1017998909.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  good_tracts['shifted_index_f'] = good_tracts['index'].shift(1)
C:\Users\zinge\AppData\Local\Temp\ipykernel_15884\1017998909.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  good_tracts['diff_f']  =  good_tracts['index'] - good_tracts['shifted_index_f']
C:\Users\zinge\AppData\Local\Temp\ipykernel_15884\1017998909.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

WP_235624429 and WP_012296277
None
WP_243820847 and WP_243820848
None


C:\Users\zinge\AppData\Local\Temp\ipykernel_15884\1017998909.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  good_tracts['shifted_index_f'] = good_tracts['index'].shift(1)
C:\Users\zinge\AppData\Local\Temp\ipykernel_15884\1017998909.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  good_tracts['diff_f']  =  good_tracts['index'] - good_tracts['shifted_index_f']
C:\Users\zinge\AppData\Local\Temp\ipykernel_15884\1017998909.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

WP_275719485 and WP_275719486
None
                    protein_names  tract ID top left  tract ID bottom right  \
3   WP_149191159 and WP_149191160                3.0                    0.0   
6   WP_185560005 and WP_185560006                2.0                    0.0   
7   WP_199404901 and WP_199404900                0.0                    0.0   
9   WP_199405059 and WP_199405058                0.0                    0.0   
11  WP_205420459 and WP_055004244                1.0                    0.0   
12  WP_226888708 and WP_226003473                0.0                    0.0   
13  WP_228302989 and WP_223575842                0.0                    0.0   
16  WP_228474820 and WP_185306028                2.0                    0.0   
17  WP_233679651 and WP_061090952                0.0                    0.0   
20  WP_235624429 and WP_012296277                2.0                    0.0   
22  WP_243820847 and WP_243820848                1.0                    0.0   
24  WP_275719485 

C:\Users\zinge\AppData\Local\Temp\ipykernel_15884\1017998909.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  good_tracts['shifted_index_f'] = good_tracts['index'].shift(1)
C:\Users\zinge\AppData\Local\Temp\ipykernel_15884\1017998909.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  good_tracts['diff_f']  =  good_tracts['index'] - good_tracts['shifted_index_f']
C:\Users\zinge\AppData\Local\Temp\ipykernel_15884\1017998909.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice